In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from pathlib import Path

OBJ_PATH = Path('0AAQ6BO3_lower.obj')
OUT_DIR  = Path('.').resolve()

# Read OBJ

In [ ]:
verts, faces = [], []
with OBJ_PATH.open('r', encoding='utf-8', errors='ignore') as f:
    for line in f:
        parts = line.strip().split()
        if not parts:
            continue
        if parts[0] == 'v' and len(parts) >= 4:
            verts.append(list(map(float, parts[1:4])))
        elif parts[0] == 'f':
            idxs = [int(tok.split('/')[0]) - 1 for tok in parts[1:]]
            if len(idxs) == 3:
                faces.append(idxs)
            elif len(idxs) > 3:
                faces += [[idxs[i] for i in tri] for tri in ([0,1,2], [0,2,3])]

verts  = np.asarray(verts, dtype=np.float32)
faces  = np.asarray(faces, dtype=np.int32)

# Some statistics are pre-calculated

In [ ]:
x_all          = verts[:, 0]
x_min, x_max   = x_all.min(), x_all.max()
x_mid          = (x_min + x_max) / 2
slice_half_w   = 0.01 * (x_max - x_min)

mask_left  = (verts[faces].max(axis=1)[:, 0] <  x_mid - slice_half_w)
mask_right = (verts[faces].min(axis=1)[:, 0] >  x_mid + slice_half_w)
mask_mid   = np.logical_not(np.logical_or(mask_left, mask_right))

FACE_SUBSETS = {
    'left_sagittal' : faces[mask_left],
    'mid_sagittal'  : faces[mask_mid],
    'right_sagittal': faces[mask_right],
}


def render_view(
    verts3d: np.ndarray,
    faces3d: np.ndarray,
    proj_axes: tuple[int, int],
    depth_axis: int,
    title: str,
    outfile: Path,
    invert_y: bool = True,
):
    proj   = verts3d[:, proj_axes]
    depth  = verts3d[:, depth_axis]
    d_min, d_max = depth.min(), depth.max()

    fig, ax = plt.subplots(figsize=(6, 6))
    ax.set_aspect('equal')
    ax.axis('off')

    for face in faces3d:
        pts2d  = proj[face]
        shade  = (depth[face].mean() - d_min) / (d_max - d_min + 1e-8)
        gray   = 1.0 - 0.5 * shade
        poly   = Polygon(
            pts2d,
            closed=True,
            facecolor=(gray, gray, gray),
            edgecolor='black',
            linewidth=0.1,
        )
        ax.add_patch(poly)

    ax.relim(); ax.autoscale_view()
    if invert_y: ax.invert_yaxis()
    ax.set_title(title, fontsize=14)
    fig.tight_layout()
    fig.savefig(outfile, dpi=300, bbox_inches='tight')
    plt.close(fig)
    print(f"save {outfile.name}")

VIEW_SPECS = [
    ('front',        (0, 1),           2,       faces),
    ('top',          (0, 2),           1,       faces),
    ('left_sagittal',  (1, 2),         0,       FACE_SUBSETS['left_sagittal']),
    ('mid_sagittal',   (1, 2),         0,       FACE_SUBSETS['mid_sagittal']),
    ('right_sagittal', (1, 2),         0,       FACE_SUBSETS['right_sagittal']),
]

OUT_DIR.mkdir(exist_ok=True)
for name, axes2d, depth_axis, fs in VIEW_SPECS:
    render_view(
        verts3d   = verts,
        faces3d   = fs,
        proj_axes = axes2d,
        depth_axis= depth_axis,
        title     = name.replace('_', ' ').title(),
        outfile   = OUT_DIR / f'{OBJ_PATH.stem}_{name}.png',
    )